<a href="https://colab.research.google.com/github/taaha3244/atom-bot/blob/main/QdrantEmbeddingsSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
!pip install langchain
!pip install --upgrade --quiet  qdrant-client
!pip install  langchain_openai


In [9]:
import gradio as gr
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http import models
from google.colab import userdata
from qdrant_client import QdrantClient
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [23]:
def atom_bot(question: str,openai_api_key):

  qdrant_client = QdrantClient(
    url=userdata.get('qdrant_end'),
    api_key=userdata.get('qdrant')
  )

  embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
  query_result = embeddings.embed_query(question)

  response = qdrant_client.search(
    collection_name="atom_bot_embeddings",
    query_vector=query_result
   # Number of closest vectors to retrieve
  )


  prompt=PromptTemplate(
    template=""""You ara a chatbot named 'Atom-bot' and belong to an online education platform named Atom camp.
    Your purpose is to spread positive image of the institute and give information to the user.Use the following pieces of context to answer the questions at the end.If
    you don't know the answer, just say don't know. do not try to make up the answer.

    <context>
    {context}
    </context>

    Question: {question}
    Helpful Answer,formatted in markdown:""",
    input_variables=["context","question"]
  )

  llm=OpenAI(openai_api_key=openai_api_key)
  chain=LLMChain(llm=llm,prompt=prompt)

  result=chain({
    "question":question,
    "context":"/n".join([doc.payload['page_content'] for doc in response])
  })

  return (result['text'])





In [15]:
atom_bot('who is Mr Naveed?')

'\n\nDr. Naveed Iftikhar is the co-founder of Atom camp, the online education platform focused on data science and AI. He is one of the distinguished speakers at the "Data Science and AI: The Science of Future" event hosted by Atom camp at NIC Karachi and is also mentioned as one of the educational partners of the institute. '

In [25]:
inputs = [
    gr.Textbox(lines=3, label="Enter your question"),
    gr.Textbox(label="OpenAI API Key", type="password" )
]
outputs = gr.Textbox(label="Response")

# Create Gradio interface
gr.Interface(
    fn=atom_bot,
    inputs=inputs,
    outputs=outputs,
    title="Atom-bot Chatbot",
    description="Ask any question related to Atom Camp online education platform.",
    examples=[["What courses does Atom Camp offer?"]]
).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://10b552623d437dfc58.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
